In [72]:
from pathlib import Path
import csv

In [73]:
p = Path(r'D:\python\some_tools\dxf_test\skb20')
cgtow = p / 'cgtow.txt'
tabs = []   # таблицы по каждой фазе

запишем лист с координатами опор из файла

In [74]:
ctw = []
with cgtow.open() as f:
    spam = csv.reader(f, delimiter='\t', skipinitialspace=True)
    for row in spam:
        if not row[1] == '':
            ctw.append(row)
ctw[1][1]

'468188.6'

In [75]:
dxf_files = list(p.glob('40*.dxf'))   # list of dxf files

создаем функцию парсер dxf

In [76]:
def dxf_poly_parse(path):
    
    dxf = []   # list to write all data from file
    
    with open(path) as dxf_file:
        for line in dxf_file:
            dxf.append(line.strip())   # write each stroke to list
            
    polylines = []   # empty list for polylines
    line = []   # list for coords of each polyline

    for i in range(len(dxf)):
        if dxf[i] == 'POLYLINE':
            line = []   # list cleaning
        if dxf[i] == 'VERTEX' and dxf[i+1] == '10':
            line.append((float(dxf[i+2]), float(dxf[i+4]), float(dxf[i+6])))
        if dxf[i] == 'SEQEND':
            polylines.append(line)
    
    return polylines

делаем функцию рассчета расстояния 2д

In [77]:
def dist(x1, y1, x2, y2):
    """ расчет расстояния """
    return (((x2-x1)**2 + (y2-y1)**2)**(1/2))

проход по дхф файлам

In [78]:
mid_points = []   # list for mid points
for file in dxf_files:
    mids = []   # list of mid points for each phase
    phase = file.stem.split('_')[-1]   # name of phase
    polylines = dxf_poly_parse(file)   # calling function to have polylines
    
    for line in range(len(polylines)):
        d1_min = d2_min = 1000000    # fake distance
        id_start = id_end = 'None'   # fake id
 
        st_x = polylines[line][0][0]
        st_y = polylines[line][0][1]
        end_x = polylines[line][-1][0]
        end_y = polylines[line][-1][1]
        
        # finding start and end ID
        for a in range(len(ctw)):
            # calc if one of other structures is closer to the start of polyline
            
            d2 = dist(st_x, st_y, float(ctw[a][1]), float(ctw[a][2]))
            
            if d2 < d1_min:
                d1_min = d2   # rewrite min dist
                id_start = ctw[a][0]   # rewrite id
                
        for b in range(len(ctw)):
            # calc if one of other structures is closer to the end of polyline
            d2 = dist(end_x, end_y, float(ctw[b][1]), float(ctw[b][2]))
            
            if d2 < d2_min:
                d2_min = d2   # rewrite min dist
                id_end = ctw[b][0]   # rewrite id
        
        mid_x = st_x - (st_x-end_x)/2
        mid_y = st_y - (st_y-end_y)/2
        
        # finding mid_z
        cv_num = len(polylines[line])//2   # central vertex number
        
        d_cv_mid = dist(polylines[line][cv_num][0], polylines[line][cv_num][1], mid_x, mid_y)
        d_b_vert = dist(polylines[line][cv_num][0], polylines[line][cv_num][1], polylines[line][cv_num+1][0], polylines[line][cv_num+1][1])
        mid_z = polylines[line][cv_num][2] - ((polylines[line][cv_num][2] - polylines[line][cv_num+1][2])*(d_cv_mid/d_b_vert))
        
        mids.append((phase, id_start, id_end, round(mid_x, 2),  round(mid_y, 2), round(mid_z, 2)))
    mid_points.append(mids)


In [79]:
import pandas as pd

In [26]:
import openpyxl

In [80]:
mp = {}
for tab in mid_points:
    mp[tab[0][0]] = pd.DataFrame(tab, columns=['phase', 'from', 'to', 'x', 'y', 'z'])

In [81]:
mp['1a']

,phase,from,to,x,y,z
0,1a,ПС Суоярви,1,468162.95,6885564.24,168.68
1,1a,1,2,468249.77,6885579.90,168.37
2,1a,2,3,468370.10,6885628.13,168.65
3,1a,3,4,468502.47,6885691.97,165.38
4,1a,4,5,468642.36,6885759.23,166.51
...,...,...,...,...,...,...
152,1a,149,150,482591.47,6907680.31,159.72
153,1a,150,151,482717.24,6907770.73,163.10
154,1a,151,152,482815.48,6907839.93,167.05
155,1a,152,153,482859.54,6907911.69,162.89


In [82]:
# del(mid_points)

In [83]:
for key in mp.keys():
    mp[key]['span'] = mp[key][['from', 'to']].agg(' - '.join, axis=1)

In [84]:
mp_ab = mp['1a'].merge(mp['1b'][['span', 'x', 'y', 'z']], left_on='span', right_on='span', suffixes=('_1a', '_1b'))

In [85]:
mp_ab

,phase,from,to,x_1a,y_1a,z_1a,span,x_1b,y_1b,z_1b
0,1a,ПС Суоярви,1,468162.95,6885564.24,168.68,ПС Суоярви - 1,468162.98,6885563.54,170.51
1,1a,1,2,468249.77,6885579.90,168.37,1 - 2,468250.45,6885578.87,170.70
2,1a,2,3,468370.10,6885628.13,168.65,2 - 3,468371.77,6885624.95,168.71
3,1a,3,4,468502.47,6885691.97,165.38,3 - 4,468504.14,6885688.73,165.66
4,1a,4,5,468642.36,6885759.23,166.51,4 - 5,468643.94,6885755.69,166.74
...,...,...,...,...,...,...,...,...,...,...
152,1a,149,150,482591.47,6907680.31,159.72,149 - 150,482593.90,6907677.37,160.52
153,1a,150,151,482717.24,6907770.73,163.10,150 - 151,482719.65,6907767.45,164.05
154,1a,151,152,482815.48,6907839.93,167.05,151 - 152,482818.53,6907837.04,167.73
155,1a,152,153,482859.54,6907911.69,162.89,152 - 153,482863.12,6907910.02,162.65


In [86]:
mp_abc = mp_ab.merge(mp['1c'][['span', 'x', 'y', 'z']], left_on='span', right_on='span', suffixes=(False, '_1c'))

In [87]:
mp_abc = mp_abc.rename(columns={'x': 'x_1c', 'y': 'y_1c', 'z': 'z_1c'})

In [88]:
mp_abc = mp_abc[['span', 'from', 'to', 'x_1a', 'y_1a', 'z_1a', 'x_1b', 'y_1b', 'z_1b', 'x_1c', 'y_1c', 'z_1c']]

In [89]:
mp_abc

,span,from,to,x_1a,y_1a,z_1a,x_1b,y_1b,z_1b,x_1c,y_1c,z_1c
0,ПС Суоярви - 1,ПС Суоярви,1,468162.95,6885564.24,168.68,468162.98,6885563.54,170.51,468163.17,6885561.72,172.59
1,1 - 2,1,2,468249.77,6885579.90,168.37,468250.45,6885578.87,170.70,468251.30,6885576.21,172.25
2,2 - 3,2,3,468370.10,6885628.13,168.65,468371.77,6885624.95,168.71,468373.34,6885621.46,169.00
3,3 - 4,3,4,468502.47,6885691.97,165.38,468504.14,6885688.73,165.66,468505.72,6885685.29,166.03
4,4 - 5,4,5,468642.36,6885759.23,166.51,468643.94,6885755.69,166.74,468645.45,6885752.10,167.17
...,...,...,...,...,...,...,...,...,...,...,...,...
152,149 - 150,149,150,482591.47,6907680.31,159.72,482593.90,6907677.37,160.52,482596.21,6907674.36,160.32
153,150 - 151,150,151,482717.24,6907770.73,163.10,482719.65,6907767.45,164.05,482722.03,6907764.09,163.78
154,151 - 152,151,152,482815.48,6907839.93,167.05,482818.53,6907837.04,167.73,482821.62,6907834.02,166.20
155,152 - 153,152,153,482859.54,6907911.69,162.89,482863.12,6907910.02,162.65,482866.71,6907908.38,162.03


In [90]:
def dist3d(x1, y1, z1, x2, y2, z2):
    """ расчет 3d расстояния """
    return round((((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)**(1/2)), 2)

In [91]:
mp_abc['dist_ab'] = dist3d(mp_abc['x_1a'], mp_abc['y_1a'], mp_abc['z_1a'], mp_abc['x_1b'], mp_abc['y_1b'], mp_abc['z_1b'])

In [92]:
mp_abc['dist_bc'] = dist3d(mp_abc['x_1b'], mp_abc['y_1b'], mp_abc['z_1b'], mp_abc['x_1c'], mp_abc['y_1c'], mp_abc['z_1c'])

In [93]:
mp_abc['dist_ac'] = dist3d(mp_abc['x_1a'], mp_abc['y_1a'], mp_abc['z_1a'], mp_abc['x_1c'], mp_abc['y_1c'], mp_abc['z_1c'])

In [94]:
mp_abc.to_csv(path_or_buf='1a.txt', index=False, columns=['x_1a', 'y_1a', 'z_1a'], sep=' ', header=False)

In [95]:
mp_abc.to_csv(path_or_buf='1b.txt', index=False, columns=['x_1b', 'y_1b', 'z_1b'], sep=' ', header=False)

In [96]:
mp_abc.to_csv(path_or_buf='1c.txt', index=False, columns=['x_1c', 'y_1c', 'z_1c'], sep=' ', header=False)

In [97]:
mp_abc.to_excel("output.xlsx")